# Contextual Compression in Document Retrieval

## Overview

This code demonstrates the implementation of contextual compression in a document retrieval system using LangChain and OpenAI's language models. The technique aims to improve the relevance and conciseness of retrieved information by compressing and extracting the most pertinent parts of documents in the context of a given query.

## Motivation

Traditional document retrieval systems often return entire chunks or documents, which may contain irrelevant information. Contextual compression addresses this by intelligently extracting and compressing only the most relevant parts of retrieved documents, leading to more focused and efficient information retrieval.

## Key Components

1. Vector store creation from a PDF document
2. Base retriever setup
3. LLM-based contextual compressor
4. Contextual compression retriever
5. Question-answering chain integrating the compressed retriever

## Method Details

### Document Preprocessing and Vector Store Creation

1. The PDF is processed and encoded into a vector store using a custom `encode_pdf` function.

### Retriever and Compressor Setup

1. A base retriever is created from the vector store.
2. An LLM-based contextual compressor (LLMChainExtractor) is initialized using OpenAI's GPT-4 model.

### Contextual Compression Retriever

1. The base retriever and compressor are combined into a ContextualCompressionRetriever.
2. This retriever first fetches documents using the base retriever, then applies the compressor to extract the most relevant information.

### Question-Answering Chain

1. A RetrievalQA chain is created, integrating the compression retriever.
2. This chain uses the compressed and extracted information to generate answers to queries.

## Benefits of this Approach

1. Improved relevance: The system returns only the most pertinent information to the query.
2. Increased efficiency: By compressing and extracting relevant parts, it reduces the amount of text the LLM needs to process.
3. Enhanced context understanding: The LLM-based compressor can understand the context of the query and extract information accordingly.
4. Flexibility: The system can be easily adapted to different types of documents and queries.

## Conclusion

Contextual compression in document retrieval offers a powerful way to enhance the quality and efficiency of information retrieval systems. By intelligently extracting and compressing relevant information, it provides more focused and context-aware responses to queries. This approach has potential applications in various fields requiring efficient and accurate information retrieval from large document collections.

<div style="text-align: center;">

<img src="../images/contextual_compression.svg" alt="contextual compression" style="width:70%; height:auto;">
</div>

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
# Install required packages
!pip install langchain python-dotenv

In [ ]:
# Clone the repository to access helper functions and evaluation modules
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# If you need to run with the latest data
# !cp -r RAG_TECHNIQUES/data .

In [1]:
import sys
from pathlib import Path

# 1. Define the directory *containing* the all_rag_techniques package
# Get the directory of the current notebook/script (__file__ might not work in some notebooks)
# Assuming the notebook is inside all_rag_techniques/
current_dir = Path.cwd() 

# The directory containing 'all_rag_techniques' is the parent directory
project_root = current_dir.parent 

# 2. Add this root to the system path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"Added project root to path: {project_root}")
else:
    print("Project root already in path.")

# 3. Now the import should work
try:
    from all_rag_techniques import setup_environment, check_keys
    print("✅ Package imported successfully!")
    setup_environment()
    check_keys()
except Exception as e:
    print(f"❌ Final import failed: {e}")

Added project root to path: /Users/ruhwang/Desktop/AI/my_projects/context-engineering/advanced-rag
✅ Package imported successfully!
LANGCHAIN_API_KEY not set (empty in .env file)
Environment setup complete!
=== API Keys from config.py ===
  GROQ_API_KEY: Loaded
  COHERE_API_KEY: Loaded
  OPENAI_API_KEY: Loaded
  LANGCHAIN_API_KEY: Missing

=== Environment Variables ===
  os.environ['GROQ_API_KEY']: Set
  os.environ['COHERE_API_KEY']: Set

All essential keys loaded!


In [2]:
import os
import sys
from dotenv import load_dotenv
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA


# Original path append replaced for Colab compatibility
from helper_functions import *
from evaluation.evalute_rag import *

/var/folders/rt/0kf7v29569z7rctzmpst_dsh0000gn/T/ipykernel_6370/956389574.py:10: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper_functions import *


### Define document's path

In [ ]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [3]:
path = "/Users/ruhwang/Desktop/AI/my_projects/context-engineering/advanced-rag/data/Understanding_Climate_Change.pdf"

### Create a vector store

In [4]:
vector_store = encode_pdf(path)

In [5]:
type(vector_store)

langchain_community.vectorstores.faiss.FAISS

### Create a retriever + contexual compressor + combine them 

In [15]:
from openai import OpenAI
from langchain_groq import ChatGroq

In [ ]:
# Create a retriever
retriever = vector_store.as_retriever()

#Create a contextual compressor
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000) # ChatGroq(model='openai/gpt-oss-20b', temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

#Combine the retriever with the compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# Create a QA chain with the compressed retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    return_source_documents=True
)

In [8]:
type(LLMChainExtractor.from_llm(llm))

langchain.retrievers.document_compressors.chain_extract.LLMChainExtractor

In [9]:
compression_retriever

ContextualCompressionRetriever(base_compressor=LLMChainExtractor(llm_chain=PromptTemplate(input_variables=['context', 'question'], input_types={}, output_parser=NoOutputParser(), partial_variables={}, template='Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT. \n\nRemember, *DO NOT* edit the extracted parts of the context.\n\n> Question: {question}\n> Context:\n>>>\n{context}\n>>>\nExtracted relevant parts:')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x169156f30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x169157a40>, root_client=<openai.OpenAI object at 0x106876210>, root_async_client=<openai.AsyncOpenAI object at 0x169156bd0>, model_name='gpt-5-nano', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=4000)
| NoOutputParser(), get_inpu

In [10]:
qa_chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x169156f30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x169157a40>, root_client=<openai.OpenAI object at 0x106876210>,

In [11]:
RetrievalQA

langchain.chains.retrieval_qa.base.RetrievalQA

### Example usage

ChatGroq (gpt-oss-20b)

In [17]:
query = "What is the main topic of the document?"
result = qa_chain.invoke({"query": query})
print(result["result"])
print("Source documents:", result["source_documents"])

The document’s main topic is **climate change**—specifically its broad impacts on human health, society, and the environment, and the policies, actions, and justice considerations needed to address those challenges.
Source documents: [Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2024-07-13T20:17:34+03:00', 'author': 'Nir', 'moddate': '2024-07-13T20:17:34+03:00', 'source': '/Users/ruhwang/Desktop/AI/my_projects/context-engineering/advanced-rag/data/Understanding_Climate_Change.pdf', 'total_pages': 33, 'page': 14, 'page_label': '15'}, page_content='Chapter 10: Climate Change and Human Health \nHealth Impacts'), Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2024-07-13T20:17:34+03:00', 'author': 'Nir', 'moddate': '2024-07-13T20:17:34+03:00', 'source': '/Users/ruhwang/Desktop/AI/my_projects/context-engineering/advanced-rag/data/Understanding_Climate_Change.pdf', 'total_

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--contextual-compression)